In [1]:
!hostname

r001.ib.bridges2.psc.edu


In [2]:
import os
from pathlib import Path
from itertools import product
import time
from typing import Tuple
from glob import glob

import duckdb
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
def dummy_calculation(df: pd.DataFrame) -> float:
    array_values = df["array"].values
    array_values = np.concatenate(array_values)
    return np.mean(array_values)

In [4]:
def run_test(
    nrows: int,
    nelem: int,
    root: str = "data",
    expected_n: int = 200,
) -> Tuple[pd.DataFrame, float, float]:

    tic = time.monotonic()
    datadir = Path(root)  / f"nrows-{nrows}_nelem-{nelem}"
    attrib = duckdb.from_parquet(str(datadir / "attrib.parquet"))
    arrays = duckdb.from_parquet(str(datadir / "arrays-*.parquet"))

    threshold = expected_n / len(attrib)
    filtered = attrib.filter(f"attrib < {threshold:.6f}")
    filtered = filtered.set_alias("filtered")
    arrays = arrays.set_alias("arrays")

    joined = filtered.join(arrays, "filtered.index = arrays.index").df()
    result = dummy_calculation(joined)
    rt = time.monotonic() - tic
    return joined, rt, result

In [5]:
def run_test_pandas(
    nrows: int,
    nelem: int,
    root: str = "data",
    expected_n: int = 200,
) -> Tuple[pd.DataFrame, float, float]:

    tic = time.monotonic()
    datadir = Path(root)  / f"nrows-{nrows}_nelem-{nelem}"
    attrib = pd.read_parquet(str(datadir / "attrib.parquet"))

    threshold = expected_n / len(attrib)
    filtered = attrib.loc[attrib["attrib"] < round(threshold, 6)]

    joined = []
    array_paths = sorted(glob(str(datadir / "arrays-*.parquet")))
    for array_path in array_paths:
        arrays = pd.read_parquet(array_path)
        joined.append(filtered.merge(arrays, on="index", how="inner"))
    joined = pd.concat(joined, ignore_index=True) 
    result = dummy_calculation(joined)
    rt = time.monotonic() - tic
    return joined, rt, result

In [6]:
nrows = 100000
nelem = 1000

joined, rt, result = run_test(nrows, nelem)

In [7]:
joined.head()

,index,attrib,index_2,array
0,96396,0.001486,96396,"[-0.9589411518757738, -0.17018504684840893, -0..."
1,96485,0.000056,96485,"[0.849472157465993, 0.28572346823373884, -1.05..."
2,96535,0.001625,96535,"[1.0960983722991202, -0.12945497660078523, 1.6..."
3,96662,0.001350,96662,"[0.45738849878008314, -1.7207773066883745, 0.4..."
4,96720,0.000357,96720,"[0.4552732291483246, 1.030644894273107, 0.2919..."


In [8]:
results_ddb = []

nrows = 10 ** np.arange(3, 6)
nelems = 10 ** np.arange(1, 5)

for nrow, nelem in product(nrows, nelems):
    df, rt, val = run_test(nrow, nelem)
    results_ddb.append([nrow, nelem, rt, val])
    print(f"nrow={nrow}, nelem={nelem}, rt={rt:.3f}s")

results_ddb = pd.DataFrame(results_ddb, columns=["nrow", "nelem", "rt", "val"])

nrow=1000, nelem=10, rt=0.007s
nrow=1000, nelem=100, rt=0.014s
nrow=1000, nelem=1000, rt=0.076s
nrow=1000, nelem=10000, rt=0.593s
nrow=10000, nelem=10, rt=0.010s
nrow=10000, nelem=100, rt=0.023s
nrow=10000, nelem=1000, rt=0.104s
nrow=10000, nelem=10000, rt=0.806s
nrow=100000, nelem=10, rt=0.023s
nrow=100000, nelem=100, rt=0.050s
nrow=100000, nelem=1000, rt=0.248s
nrow=100000, nelem=10000, rt=2.998s


In [9]:
results_pd = []

for nrow, nelem in product(nrows, nelems):
    df, rt, val = run_test_pandas(nrow, nelem)
    results_pd.append([nrow, nelem, rt, val])
    print(f"nrow={nrow}, nelem={nelem}, rt={rt:.3f}s")

results_pd = pd.DataFrame(results_pd, columns=["nrow", "nelem", "rt", "val"])

nrow=1000, nelem=10, rt=0.120s
nrow=1000, nelem=100, rt=0.009s
nrow=1000, nelem=1000, rt=0.019s
nrow=1000, nelem=10000, rt=0.123s
nrow=10000, nelem=10, rt=0.012s
nrow=10000, nelem=100, rt=0.021s
nrow=10000, nelem=1000, rt=0.111s
nrow=10000, nelem=10000, rt=1.045s
nrow=100000, nelem=10, rt=0.056s
nrow=100000, nelem=100, rt=0.148s
nrow=100000, nelem=1000, rt=1.065s
nrow=100000, nelem=10000, rt=10.339s


In [10]:
np.allclose(results_ddb["val"], results_pd["val"])

True

In [11]:
results = pd.concat([results_ddb.loc[:, ["nrow", "nelem", "rt"]], results_pd.loc[:, ["rt"]]], axis=1)
results.columns = ["nrow", "nelem", "runtime (DuckDB)", "runtime (pandas)"]

In [12]:
print(results.to_markdown(index=False))

|   nrow |   nelem |   runtime (DuckDB) |   runtime (pandas) |
|-------:|--------:|-------------------:|-------------------:|
|   1000 |      10 |         0.00729267 |         0.119576   |
|   1000 |     100 |         0.0136764  |         0.00939533 |
|   1000 |    1000 |         0.0761871  |         0.019083   |
|   1000 |   10000 |         0.593045   |         0.12254    |
|  10000 |      10 |         0.0097008  |         0.011616   |
|  10000 |     100 |         0.0230262  |         0.0214925  |
|  10000 |    1000 |         0.103769   |         0.110753   |
|  10000 |   10000 |         0.806461   |         1.04455    |
| 100000 |      10 |         0.0229186  |         0.0559437  |
| 100000 |     100 |         0.0502371  |         0.147699   |
| 100000 |    1000 |         0.248135   |         1.06513    |
| 100000 |   10000 |         2.99795    |        10.3386     |
